<a href="https://colab.research.google.com/github/efecte8/funcraft/blob/main/funcraft_backend_3_1_sdxl_lcm_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

make sure to run on gpu runtime

license



In [ ]:
#Copyright 2023 funcraft

#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at

#    http://www.apache.org/licenses/LICENSE-2.0

#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

#Install requirements ~1min


*   APP API - FLASK, NGROK (tunnel to local)
*   ML - DIFFUSERS, TRANSFORMERS, ACCELERATE, PEFT



ML dependencies

In [1]:
!pip3 install --quiet --upgrade diffusers transformers accelerate peft
!pip3 install --quiet flask
!pip3 install --quiet flask-ngrok pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 15.8 MB/s eta 0:00:00


# Model Pipeline:  sdxl1.0 LCM Lora ~2min

In [ ]:
import torch
from diffusers import LCMScheduler, DiffusionPipeline, AutoPipelineForText2Image, AutoPipelineForInpainting, AutoPipelineForImage2Image

base_pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    variant="fp16",
    torch_dtype=torch.float16
).to("cuda")

# set scheduler
base_pipe.scheduler = LCMScheduler.from_config(base_pipe.scheduler.config)

# load LoRAs
base_pipe.load_lora_weights("latent-consistency/lcm-lora-sdxl", adapter_name="lcm")
base_pipe.fuse_lora()
generator = torch.manual_seed(0)
base_pipe.generator = generator



In [ ]:
img2img_pipe= AutoPipelineForImage2Image.from_pipe(base_pipe)
img2img_pipe.to("cuda")

In [ ]:
inpainting_pipe = AutoPipelineForInpainting.from_pipe(base_pipe)
inpainting_pipe.to("cuda")

In [16]:
#text to image function, check for generator as well for seed
def text_to_image(prompt, negative_prompt, num_inference_steps=4, guidance_scale=1):
  image0 = base_pipe(prompt=prompt, negative_prompt=negative_prompt, guidance_scale=guidance_scale,num_inference_steps=num_inference_steps ).images[0]
  torch.cuda.empty_cache()
  return image0

#image to image function, check for generator as well for seed
def image_to_image(prompt, negative_prompt, image, strength=0.8, num_inference_steps=4, guidance_scale=1):
  image1=img2img_pipe(prompt, negative_prompt=negative_prompt, image=image, strength=strength, guidance_scale=guidance_scale, num_inference_steps=num_inference_steps).images[0]
  torch.cuda.empty_cache()
  return image1

#inpainting
def image_inpainting(prompt, negative_prompt, image, mask_image, strength=0.7, num_inference_steps=4, guidance_scale=1):
  image2= inpainting_pipe(prompt=prompt, negative_prompt=negative_prompt, image=image, mask_image=mask_image, strength=strength, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]
  torch.cuda.empty_cache()
  return image2

#APP and API
- create a web app backend that sends the images to the local using a ngroc url
- create the api routes
- run the app and start listening for requests

In [28]:
from flask import Flask, request, send_file
from PIL import Image
import io
import zipfile

from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route('/inpainting', methods=['POST'])
def process_image2():
    prompt = str(request.form['prompt'])
    gen_mode = int(request.form['gen_mode'])
    guidance_scale = int(request.form['guidance_scale'])
    strength_scale = float(request.form['strength_scale'])
    number_of_steps = int(request.form['number_of_steps'])
    seed = int(request.form['seed'])
    negative_prompt= str(request.form['negative_prompt'])

    print(f'Prompt: {prompt}')
    print(f'Gen Mode: {gen_mode}')
    print(f'Guidance Scale: {guidance_scale}')
    print(f'Strength Scale: {strength_scale}')
    print(f'Number of Steps: {number_of_steps}')
    print(f'Seed: {seed}')
    print(f'Negative prompt: {negative_prompt}')

    image_file = request.files['image']  # Get the uploaded image file
    mask_image_file = request.files['mask_image']

    # Convert the image files to PIL Image objects
    image = Image.open(image_file.stream)
    mask_image = Image.open(mask_image_file.stream)
    #display(image)
    #display(mask_image)
    image_inpaint= image_inpainting(prompt=prompt, negative_prompt=negative_prompt, image=image, mask_image=mask_image, strength=strength_scale, num_inference_steps=number_of_steps, guidance_scale=guidance_scale)
    #display(image_inpaint)

    # Convert the PIL Image to bytes
    image_inpaint_bytes = io.BytesIO()
    image_inpaint.save(image_inpaint_bytes, format='PNG')
    image_inpaint_bytes.seek(0)

    # Return the processed image as a response
    return send_file(image_inpaint_bytes, mimetype='image/png')

@app.route('/imagetoimage', methods=['POST'])
def process_image1():
    prompt = str(request.form['prompt'])
    gen_mode = int(request.form['gen_mode'])
    guidance_scale = int(request.form['guidance_scale'])
    strength_scale = float(request.form['strength_scale'])
    number_of_steps = int(request.form['number_of_steps'])
    seed = int(request.form['seed'])
    negative_prompt= str(request.form['negative_prompt'])
    selected_style=str(request.form['selected_style'])

    print(f'Prompt: {prompt}')
    print(f'Gen Mode: {gen_mode}')
    print(f'Guidance Scale: {guidance_scale}')
    print(f'Strength Scale: {strength_scale}')
    print(f'Number of Steps: {number_of_steps}')
    print(f'Seed: {seed}')
    print(f'Negative prompt: {negative_prompt}')
    print(f'Selected style : {selected_style}')

    if selected_style != 'No Style':
      prompt, negative_prompt= style_modifier(input_prompt=prompt, input_neg_prompt=negative_prompt, style=selected_style)



    image_file = request.files['image']  # Get the uploaded image file

    # Convert the image files to PIL Image objects
    image = Image.open(image_file.stream)
    #display(image)
    image_imagetoimage= image_to_image(prompt=prompt, negative_prompt=negative_prompt, image=image, strength=strength_scale, num_inference_steps=number_of_steps, guidance_scale=guidance_scale)
    #display(image_imagetoimage)

    # Convert the PIL Image to bytes
    imagetoimage_bytes = io.BytesIO()
    image_imagetoimage.save(imagetoimage_bytes, format='PNG')
    imagetoimage_bytes.seek(0)

    # Return the processed image as a response
    return send_file(imagetoimage_bytes, mimetype='image/png')


@app.route('/texttoimage', methods=['POST'])
def process_image0():
    prompt = str(request.form['prompt'])
    gen_mode = int(request.form['gen_mode'])
    guidance_scale = int(request.form['guidance_scale'])
    strength_scale = float(request.form['strength_scale'])
    number_of_steps = int(request.form['number_of_steps'])
    seed = int(request.form['seed'])
    negative_prompt= str(request.form['negative_prompt'])
    selected_style=str(request.form['selected_style'])

    print(f'Prompt: {prompt}')
    print(f'Gen Mode: {gen_mode}')
    print(f'Guidance Scale: {guidance_scale}')
    print(f'Strength Scale: {strength_scale}')
    print(f'Number of Steps: {number_of_steps}')
    print(f'Seed: {seed}')
    print(f'Negative prompt: {negative_prompt}')
    print(f'Selected style : {selected_style}')

    if selected_style != 'No Style':
      prompt, negative_prompt= style_modifier(input_prompt=prompt, input_neg_prompt=negative_prompt, style=selected_style)


    image_texttoimage= text_to_image(prompt=prompt, negative_prompt=negative_prompt, num_inference_steps=number_of_steps, guidance_scale=guidance_scale)
    #display(image_texttoimage)

    # Convert the PIL Image to bytes
    image_texttoimage_bytes = io.BytesIO()
    image_texttoimage.save(image_texttoimage_bytes, format='PNG')
    image_texttoimage_bytes.seek(0)

    # Return the processed image as a response
    return send_file(image_texttoimage_bytes, mimetype='image/png')



if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://b52e-35-233-219-242.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Prompt: a super car with green neon back lights, parked infront of a gas station, dim lights, cannon 8k, hyper realistic
Gen Mode: 1
Guidance Scale: 0
Strength Scale: 0.8
Number of Steps: 4
Seed: 0
Negative prompt: 
Selected style : Photorealistic


  0%|          | 0/3 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 18:59:14] "POST /imagetoimage HTTP/1.1" 200 -


...

  0%|          | 0/6 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 18:59:44] "POST /imagetoimage HTTP/1.1" 200 -


Prompt: a super car with green neon back lights, parked infront of a gas station, dim lights, cannon 8k, hyper realistic
Gen Mode: 1
Guidance Scale: 0
Strength Scale: 0.8
Number of Steps: 8
Seed: 0
Negative prompt: 
Selected style : Futuristic


  0%|          | 0/6 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 19:01:00] "POST /imagetoimage HTTP/1.1" 200 -
